In [53]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import os
def get_tiff_metadata(tiff_path):
    """
    Extract metadata such as width, height, transform, and CRS from a reference TIFF file.
    """
    try:
        with rasterio.open(tiff_path) as src:
            width = src.width
            height = src.height
            transform = src.transform  # Spatial transformation matrix
            crs = src.crs  # Coordinate Reference System (CRS)
        return width, height, transform, crs
    except Exception as e:
        print(f"Error reading TIFF file: {tiff_path} - {e}")
        return None, None, None, None
    
def create_tiff_from_shapefile(input_shapefile, output_tiff, reference_tiff):
    # Load the shapefile
    shapes = gpd.read_file(input_shapefile)

    # Check for valid geometries
    if shapes.geometry.isnull().all():
        print(f"No geometries found in the shapefile: {input_shapefile}")
        return

    # Get metadata from the reference TIFF
    width, height, transform, crs = get_tiff_metadata(reference_tiff)

    if not all([width, height, transform, crs]):
        print(f"Invalid reference TIFF file: {reference_tiff}")
        return

    # Filter valid geometries
    valid_geometries = shapes.geometry.notnull() & shapes.geometry.is_valid
    shapes = shapes[valid_geometries]

    if shapes.empty:
        print(f"No valid geometries found after filtering in: {input_shapefile}")
        return

    # Rasterize the shapefile geometries to match the reference TIFF's grid
    rasterized_output = rasterize(
        [(geom, 1) for geom in shapes.geometry],  # Assign value of 1 to geometries
        out_shape=(height, width),                # Same dimensions as the reference TIFF
        fill=0,                                   # Fill empty areas with 0
        transform=transform,                      # Use the same transformation as reference TIFF
        dtype='int32'                             # Output data type
    )

    # Save the rasterized output to a TIFF file with the same spatial properties
    with rasterio.open(
        output_tiff, 'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,                                  # Single-band raster
        dtype=str(rasterized_output.dtype),
        crs=crs,                                  # Use the same CRS as reference
        transform=transform,                      # Use the same transform as reference
    ) as dst:
        dst.write(rasterized_output, 1)

    print(f"TIFF file created successfully: {output_tiff}")


# input_shp_path=os.path.join(os.getcwd(),'label data','HoangHoa_MONTH_08_0_0_0_residential.shp')
# output_tiff_path=os.path.join(os.getcwd(),'output','HoangHoa_MONTH_08_0_0_0_residential.tif')
# reference_tiff_path=os.path.join(os.getcwd(),'label data','HoangHoa_MONTH_08_0_0_0.tif')

# create_tiff_from_shapefile( input_shp_path, output_tiff_path, reference_tiff_path)


In [62]:
current_dir=os.getcwd()
label_path=os.listdir(os.path.join(current_dir,'label_data'))
for file in label_path:
    if file.endswith('.shp'):
        if "rice" not in file:
            reference_tif_file_str = "_".join(file.split('_')[:-1])+'.tif'
        else:
            reference_tif_file_str = "_".join(file.split('_')[:-2])+'.tif'
        shapefile_path = os.path.join(current_dir,"label_data", file)
        output_tiff_path = os.path.join(current_dir, "output", file.replace('.shp', '.tif'))
        reference_tiff_path = os.path.join(current_dir, "label_data", reference_tif_file_str)

        create_tiff_from_shapefile(shapefile_path, output_tiff_path, reference_tiff_path)
        print(f"Created mask for {reference_tif_file_str}.")

TIFF file created successfully: c:\Users\hung\Downloads\Downloads\output\QuanSon_MONTH_01_1_0_6_bamboo.tif
Created mask for QuanSon_MONTH_01_1_0_6.tif.
TIFF file created successfully: c:\Users\hung\Downloads\Downloads\output\QuanSon_MONTH_01_1_0_6_forest.tif
Created mask for QuanSon_MONTH_01_1_0_6.tif.
Error reading TIFF file: c:\Users\hung\Downloads\Downloads\label_data\QuanSon_MONTH_01_1_0_6_rice.tif - c:/Users/hung/Downloads/Downloads/label_data/QuanSon_MONTH_01_1_0_6_rice.tif: No such file or directory
Invalid reference TIFF file: c:\Users\hung\Downloads\Downloads\label_data\QuanSon_MONTH_01_1_0_6_rice.tif
Created mask for QuanSon_MONTH_01_1_0_6_rice.tif.
No geometries found in the shapefile: c:\Users\hung\Downloads\Downloads\label_data\QuanSon_MONTH_01_1_0_6_unidentifiable.shp
Created mask for QuanSon_MONTH_01_1_0_6.tif.
TIFF file created successfully: c:\Users\hung\Downloads\Downloads\output\QuanSon_MONTH_01_1_1_7_bamboo.tif
Created mask for QuanSon_MONTH_01_1_1_7.tif.
TIFF file 